# **Template for Torch-Pruning**

This template is just built for your convinience.

You are not required to follow the steps and method given below.

In [ ]:
!pip install --upgrade torch_pruning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/410.6 MB 127.8 MB/s eta 0:00:03

In [11]:
import torch
import torchvision
from torchvision.models import mobilenet_v2
import torch_pruning as tp
import torch.nn as nn
import numpy as np
import copy
from matplotlib import pyplot as plt
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm

##A Minimal Example ##  
In this section, you will perform channel pruning using the library [Torch-Pruning](https://github.com/VainF/Torch-Pruning).  

The puuner in Torch-Pruning has three main functions: sparse training (optional), importance estimation, and parameter removal.  
Torch-pruning offers two core features to support this process:

tp.importance(): This criteria is utilized to measure the importance of weights.  

tp.pruner(): This is a pruner used for the actual pruning of the parameters.  

For detailed information on this process, please refer to this [tutorial](https://github.com/VainF/Torch-Pruning/wiki/4.-High%E2%80%90level-Pruners/). Additionally, a more practical example is available in [here](https://github.com/VainF/Torch-Pruning/blob/master/benchmarks/main.py).

### 1. Load model


In [45]:
model = torch.load('./mobilenetv2_0.963.pth', map_location="cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [61]:
def recover_model():
  from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
  model = mobilenet_v2()

  NUM_CLASSES=10
  model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
  model = torch.load("mobilenetv2_0.963.pth")
  model.cuda()

  return model

### 2. Prepare a pruner
By default, Torch-Pruning will automatically prune the last non-singleton dim of these parameters. If you want to customize this behaviour, please provide an `unwrapped_parameters` list as the following example.

In [46]:
def train(
  model: nn.Module,
  dataloader: DataLoader,
  criterion: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  callbacks = None
) -> None:
  model.train()

  for inputs, targets in tqdm(dataloader, desc='train', leave=False):
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Reset the gradients (from the last iteration)
    optimizer.zero_grad()

    # Forward inference
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward propagation
    loss.backward()

    # Update optimizer and LR scheduler
    optimizer.step()
    scheduler.step()

    if callbacks is not None:
        for callback in callbacks:
            callback()

In [47]:
@torch.inference_mode()
def evaluate(
  model: nn.Module,
  dataloader: DataLoader,
  verbose=True,
) -> float:
  model.eval()

  num_samples = 0
  num_correct = 0

  for inputs, targets in tqdm(dataloader, desc="eval", leave=False,
                              disable=not verbose):
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Inference
    outputs = model(inputs)

    # Convert logits to class indices
    outputs = outputs.argmax(dim=1)

    # Update metrics
    num_samples += targets.size(0)
    num_correct += (outputs == targets).sum()

  return (num_correct / num_samples * 100).item()

In [48]:
transforms = {
    "train": Compose([
      Resize((224, 224)),
      ToTensor(),
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    "test": Compose([
      Resize((224, 224)),
      ToTensor(),
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

dataset = {}
for split in ["train", "test"]:
  dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
  )

# You can apply your own batch_size
dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=64,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
    drop_last=True
  )

Files already downloaded and verified
Files already downloaded and verified


In [130]:
model = recover_model()

In [112]:
sparsity_dict = {
##################### YOUR CODE STARTS HERE #####################
    # please modify the sparsity value of each layer
    # please DO NOT modify the key of sparsity_dict
    model.features[0][0] : 0,
    model.features[1].conv[0][0]: 0,
    model.features[1].conv[1]: 0,
    model.features[2].conv[0][0]: 0,
    model.features[2].conv[1][0]: 0,
    model.features[2].conv[2]: 0,
    model.features[3].conv[0][0]: 0,
    model.features[3].conv[1][0]: 0,
    model.features[3].conv[2]: 0,
    model.features[4].conv[0][0]: 0,
    model.features[4].conv[1][0]: 0,
    model.features[4].conv[2]: 0,
    model.features[5].conv[0][0]: 0,
    model.features[5].conv[1][0]: 0,
    model.features[5].conv[2]: 0,
    model.features[6].conv[0][0]: 0,
    model.features[6].conv[1][0]: 0,
    model.features[6].conv[2]: 0,
    model.features[7].conv[0][0]: 0,
    model.features[7].conv[1][0]: 0,
    model.features[7].conv[2]: 0,
    model.features[8].conv[0][0]: 0,
    model.features[8].conv[1][0]: 0,
    model.features[8].conv[2]: 0,
    model.features[9].conv[0][0]: 0,
    model.features[9].conv[1][0]: 0,
    model.features[9].conv[2]: 0.3,
    model.features[10].conv[0][0]: 0.3,
    model.features[10].conv[1][0]: 0,
    model.features[10].conv[2]: 0.3,
    model.features[11].conv[0][0]: 0.3,
    model.features[11].conv[1][0]: 0,
    model.features[11].conv[2]: 0.3,
    model.features[12].conv[0][0]: 0.3,
    model.features[12].conv[1][0]: 0,
    model.features[12].conv[2]: 0.3,
    model.features[13].conv[0][0]: 0.3,
    model.features[13].conv[1][0]: 0,
    model.features[13].conv[2]: 0.3,
    model.features[14].conv[0][0]: 0.3,
    model.features[14].conv[1][0]: 0,
    model.features[14].conv[2]: 0.15,
    model.features[15].conv[0][0]: 0.15,
    model.features[15].conv[1][0]: 0,
    model.features[15].conv[2]: 0.15,
    model.features[16].conv[0][0]: 0.15,
    model.features[16].conv[1][0]: 0,
    model.features[16].conv[2]: 0.15,
    model.features[17].conv[0][0]: 0.15,
    model.features[17].conv[1][0]: 0,
    model.features[17].conv[2]: 0.05,
    model.features[18][0]: 0.05,
    model.classifier[1]: 0
##################### YOUR CODE ENDS HERE #######################
}

In [124]:
# get the number of group in conv2d or group norm layer in the model
def get_group_num(model):
    group = {}
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.GroupNorm)) and module.groups > 1:
            group[name] = module.groups
    return group
channel_groups = get_group_num(model)

In [131]:
# Importance criterion
imp = tp.importance.GroupNormImportance(p=1) # or GroupTaylorImportance(), GroupHessianImportance(), etc.

# Initialize a pruner with the model and the importance criterion
example_inputs = torch.randn(1, 3, 224, 224).to(device)

ignored_layers = []
for m in model.modules():
  if isinstance(m, torch.nn.Linear) and m.out_features == 10: # ignore the classifier
    ignored_layers.append(m)
pruner = tp.pruner.GroupNormPruner ( # you can choose any pruner you like.
    model,
    example_inputs,
    importance=imp,   # Importance Estimator
    global_pruning=False,
    pruning_ratio=0.6, # remove 50% channels, ex :ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    iterative_steps = 1,  # number of steps to achieve the target ch_sparsity.
    ignored_layers = ignored_layers,        # ignore some layers such as the finall linear classifier
    channel_groups = channel_groups,  # round channels
)

### 3. Prune the model

In [132]:
# Model size before pruning
base_macs, base_nparams = tp.utils.count_ops_and_params(model, torch.randn(1,3,224,224).cuda())

model.eval()

if isinstance(imp, tp.importance.GroupTaylorImportance):
  # Taylor expansion requires gradients for importance estimation
  loss = model(example_inputs).sum() # A dummy loss, please replace this line with your loss function and data!
  loss.backward() # before pruner.step()

# prune
pruner.step()


# Parameter & MACs Counter
pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(model, torch.randn(1,3,224,224).cuda())
MFLOPs = pruned_macs/1e6
print("The pruned model:")
print(model)
print("Summary:")
print("MFLOPs: ")
print(MFLOPs)



The pruned model:
MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=12, bias=False)
          (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(12, 6, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(6, 38, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): Batch

In [133]:

accuracy = evaluate(model, dataloader['test'])
print(f"Accuracy before fine-tuning: {accuracy:.2f}%")

# finetune the pruned model here
# finetune(model)
num_finetune_epochs = 10
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_finetune_epochs)
criterion = nn.CrossEntropyLoss()

best_sparse_model_checkpoint = dict()
best_accuracy = 0
print(f'Finetuning Fine-grained Pruned Sparse Model')
for epoch in range(num_finetune_epochs):
    # At the end of each train iteration, we have to apply the pruning mask
    #    to keep the model sparse during the training
    train(model, dataloader['train'], criterion, optimizer, scheduler)
    accuracy = evaluate(model, dataloader['test'])
    is_best = accuracy > best_accuracy
    if is_best:
        best_sparse_model_checkpoint['state_dict'] = copy.deepcopy(model.state_dict())
        best_accuracy = accuracy
    print(f'    Epoch {epoch+1} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%')

# Parameter & MACs Counter
accuracy = evaluate(model, dataloader['test'])
print(f"Accuracy after fine-tuning: {accuracy:.2f}%")

eval:   0%|          | 0/156 [00:00<?, ?it/s]

Accuracy before fine-tuning: 10.02%
Finetuning Fine-grained Pruned Sparse Model


train:   0%|          | 0/781 [00:00<?, ?it/s]

eval:   0%|          | 0/156 [00:00<?, ?it/s]

    Epoch 1 Accuracy 65.09% / Best Accuracy: 65.09%


...

train:   0%|          | 0/781 [00:00<?, ?it/s]

eval:   0%|          | 0/156 [00:00<?, ?it/s]

    Epoch 10 Accuracy 84.61% / Best Accuracy: 85.97%


eval:   0%|          | 0/156 [00:00<?, ?it/s]

Accuracy after fine-tuning: 84.61%
